<a href="https://colab.research.google.com/github/jeonso1115/Colab/blob/main/3_t5_model_train_faq_colab_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#구글드라이브
!ls "/content/drive/My Drive/Colab Notebooks"

3.t5_model_train_faq_colab.py


In [1]:
#깃헙
!git clone https://github.com/jeonso1115/Colab.git

Cloning into 'Colab'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 123.78 KiB | 4.76 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
!ls "/content/Colab"

3_t5_model_train_faq_colab_py.ipynb  faq_database.db  similar_questions_classified.xlsx


In [ ]:
!git config --global user.name "jeonso1115"
!git config --global user.email "jeonso1115@naver.com"

In [ ]:
!git commit -m "first commit"

On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	Colab Notebooks/
	Colab/
	drive/
	faq_database.db
	sample_data/
	similar_questions_classified.xlsx

nothing added to commit but untracked files present (use "git add" to track)


In [6]:
!pip install torch torchvision torchaudio  # PyTorch 및 오디오 관련 패키지
!pip install transformers datasets  # Hugging Face 라이브러리
!pip install konlpy  # 한국어 형태소 분석기
!pip install selenium beautifulsoup4 requests  # 웹 크롤링 및 스크래핑
!pip install flask  # 웹 애플리케이션용 Flask
!pip install scikit-learn pandas spacy tensorflow  # 데이터 분석 및 머신러닝 관련 패키지
!pip install ray[tune]  # Ray Tune (하이퍼파라미터 튜닝)
!pip install torch_optimizer  # AdamW 등 추가적인 옵티마이저 지원
!pip install tensorflow  # TensorFlow (사용자 코드에서 일부 사용)
!pip install matplotlib-venn
!python -m spacy download ko_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 67.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import sys
import sqlite3
import requests
import spacy
from spacy.lang.ko.examples import sentences
import pandas as pd
from bs4 import BeautifulSoup
# 자연어 처리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, EncoderDecoderConfig, DataCollatorForSeq2Seq, pipeline, get_scheduler
# 웹 자동화
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
# 웹 애플리케이션
from flask import Flask, request, jsonify
# 딥러닝
import torch
from torch.utils.data import DataLoader, Dataset
import torch_optimizer as optim  # torch_optimizer 라이브러리 임포트
from torch.optim import AdamW
from transformers import get_scheduler
from datasets import Dataset, concatenate_datasets, DatasetDict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import ray.train
from ray.tune import Checkpoint
import uuid
from ray.tune import ExperimentAnalysis

#os.environ["TRANSFORMERS_OFFLINE"] = "1"
#os.environ['RAY_memory_usage_threshold'] = '0.85'  # 메모리 사용 임계값을 85%로 설정
#os.environ['RAY_memory_monitor_refresh_ms'] = '0'  # 메모리 모니터링 비활성화

In [ ]:
from google.colab import files
uploaded = files.upload()  # 파일 업로드 창이 뜸

Saving faq_database.db to faq_database.db


In [11]:
#0 : unnamed
#1 : id
#2 : question
#3 : url
#4 : answer
import spacy
from spacy.lang.ko.examples import sentences

model = T5ForConditionalGeneration.from_pretrained('digit82/kolang-t5-base')
tokenizer = T5Tokenizer.from_pretrained('digit82/kolang-t5-base')
nlp = spacy.load("ko_core_news_sm")

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


# 1. DB에서 데이터 불러오기
def load_faq_data():
    conn = sqlite3.connect("/content/Colab/faq_database.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM faq")
    faq_data = cursor.fetchall()
    conn.close()

    if not faq_data:
        raise ValueError("FAQ 데이터가 비어 있습니다. 데이터를 확인하세요!")

    global faq_ids, faq_questions, faq_answers, faq_context
    faq_ids = [row[1] for row in faq_data]
    # spaCy 전처리 적용
    faq_questions = [preprocess_text(row[2]) for row in faq_data]
    faq_answers = [preprocess_text(row[4]) for row in faq_data]

    faq_context = "\n".join([f"Q: {faq_questions[i]} A: {faq_answers[i]}" for i in range(len(faq_questions))])

load_faq_data()


# 2. 유사질문 분류 파일 로드 (xlsx)
similarity_file = pd.read_excel('/content/Colab/similar_questions_classified.xlsx')

# 유사질문 데이터에서 'category' 값 추출
similarity_data = []
for _, row in similarity_file.iterrows():
    question1 = preprocess_text(row['Question1'])
    question2 = preprocess_text(row['Question2'])
    label = row['Category']  # '유사' 또는 '다르다' 라벨

    similarity_data.append((question1, question2, label))

# 유사질문 데이터를 학습 데이터로 변환
similarity_dataset = {'question1': [item[0] for item in similarity_data],
                      'question2': [item[1] for item in similarity_data],
                      'label': [item[2] for item in similarity_data]}

similarity_dataset = Dataset.from_dict(similarity_dataset)



def preprocess_similarity_function(examples):
    inputs = ["질문1: " + q1 + " 질문2: " + q2 for q1, q2 in zip(examples['question1'], examples['question2'])]
    targets = list(examples['label'])  # 유사하거나 다르다는 라벨을 정답으로 사용
    targets = [str(label) for label in targets]  # 라벨을 문자열로 변환
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# FAQ 데이터셋 전처리 함수
def preprocess_faq_function(examples):
    inputs = examples['question']
    targets = examples['answer']

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


similarity_dataset = similarity_dataset.map(preprocess_similarity_function, batched=True)

faq_data = {'question': faq_questions, 'answer': faq_answers}
faq_dataset = Dataset.from_dict(faq_data)
faq_dataset = faq_dataset.map(preprocess_faq_function, batched=True)

full_dataset = concatenate_datasets([faq_dataset, similarity_dataset])
full_dataset = full_dataset.remove_columns('label') #label컬럼은 similarity_dataset 에만 있음

train_test_split_ratio = 0.2  # 검증 데이터 비율 (20%)
split_index = int(len(full_dataset) * (1 - train_test_split_ratio))

train_dataset = full_dataset.select(range(split_index))
eval_dataset = full_dataset.select(range(split_index, len(full_dataset)))



Map:   0%|          | 0/329 [00:00<?, ? examples/s]

Map:   0%|          | 0/173 [00:00<?, ? examples/s]

In [16]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

# 하이퍼파라미터 검색 공간 정의
search_space = {
    "learning_rate": tune.loguniform(5e-5, 5e-4), # loguniform(최소값, 최대값)
    "batch_size": tune.choice([2, 4, 8]),
    "weight_decay": tune.uniform(0.0, 0.03),  # 0.0에서 0.1 사이의 값을 사용
    "optimizer": tune.choice(["adamw_torch"]),
}

# 스케줄러 설정 (조기 종료 및 최적화)
scheduler = ASHAScheduler(
    metric="eval_loss",
    mode="min",
    max_t=15,  # 최대 학습 에폭 수
    grace_period=2,  # 최소 에폭 수
    reduction_factor=2
)

def generate_trial_id():
    global trial_id
    trial_id = str(uuid.uuid4())[:8]
    print(f"Generated Trial ID: {trial_id}")



def tune_model(config, model, train_dataset, eval_dataset, tokenizer):
    generate_trial_id()
    print(f"Trial ID: {trial_id}, Config: {config}")  # trial_id를 출력
    output_dir = f"/content/Raytune/checkpoints/trial_{trial_id}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        warmup_steps=500,
        weight_decay=config["weight_decay"],
        logging_dir=f"{output_dir}/logs",
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=config["learning_rate"],
        optim=config["optimizer"],
        save_total_limit=None,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        dataloader_num_workers=0,  # 멀티프로세싱 비활성화
    )

    print(f"Training arguments configured with output_dir: {output_dir}")
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

    try:
        print("모델 학습 시작")  # 디버깅 메시지
        trainer.train()
        eval_results = trainer.evaluate()
        print("평가 완료:", eval_results)
        eval_loss_value = eval_results.get("eval_loss", float("inf"))
        trainer.save_model(output_dir)
        checkpoint = Checkpoint.from_directory(output_dir)
        tune.report({"eval_loss": eval_loss_value}, checkpoint=checkpoint) # 딕셔너리 형태로 전달

    except Exception as e:
        print(f"Error during training: {e}")
        raise e


2025-04-03 01:27:13,221	INFO worker.py:1852 -- Started a local Ray instance.


In [13]:
import torch
import multiprocessing

print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
print(f"사용 가능한 CPU 개수: {multiprocessing.cpu_count()}")

사용 가능한 GPU 개수: 1
사용 가능한 CPU 개수: 2


In [14]:
import torch

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("cuDNN 사용 가능 여부:", torch.backends.cudnn.enabled)


PyTorch 버전: 2.6.0+cu124
CUDA 사용 가능 여부: True
CUDA 버전: 12.4
cuDNN 사용 가능 여부: True


In [ ]:
!pip uninstall torch -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 395.1/417.9 MB 22.5 MB/s eta 0:00:02

In [ ]:
#모델학습
analysis = tune.run(
    tune.with_parameters(tune_model, model=model, tokenizer=tokenizer, train_dataset=train_dataset, eval_dataset=eval_dataset),
    config=search_space,
    num_samples=100,  # model 개수
    scheduler=scheduler,
    resources_per_trial={"cpu": 2, "gpu": 1},  # 리소스 할당 (메모리 제한)
    max_concurrent_trials=1,  # 한 번에 실행할 모델 수,
    log_to_file="/content/Raytune/logs/train.log",
    storage_path="/content/Raytune"
)

#experiment_dir = analysis.trials[0].logdir
#analysis_check = ExperimentAnalysis(experiment_dir)

for trial in analysis_check.trials:
    print(f"Trial: {trial.trial_id}, Status: {trial.status}, Last Result: {trial.last_result}")


best_trial = analysis.get_best_trial(metric="eval_loss", mode="min")
if best_trial:
    print("Best config:", best_trial.config)

    # Get the best checkpoint
    best_checkpoint = analysis.get_best_checkpoint(best_trial, metric="eval_loss", mode="min")

    if best_checkpoint:
        best_model = T5ForConditionalGeneration.from_pretrained(best_checkpoint.to_directory())
        best_model.save_pretrained("/content/Colab/Raytune_best")
        tokenizer.save_pretrained("/content/Colab/Raytune_best")
    else:
        print("No best checkpoint found.")
else:
    print("No best trial found.")

+-------------------------------------------------------------------+
| Configuration for experiment     tune_model_2025-04-03_01-29-22   |
+-------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator            |
| Scheduler                        AsyncHyperBandScheduler          |
| Number of trials                 100                              |
+-------------------------------------------------------------------+

View detailed results here: /content/Raytune/tune_model_2025-04-03_01-29-22
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-04-03_01-27-09_925491_185/artifacts/2025-04-03_01-29-22/tune_model_2025-04-03_01-29-22/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-04-03 01:29:22. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------------------------------------------------